In [ ]:
pip show beautifulsoup4



In [7]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import urllib3

In [ ]:
http = urllib3.PoolManager()
baseUrl = 'https://overthecap.com/positional-spending#y'
search_list = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

for search_yr in search_list :
    url = 'https://overthecap.com/positional-spending#y' + search_yr
    html = http.request('GET', url).data.decode('utf-8')
    soup = BeautifulSoup(html, features='lxml')
    #Specify table name which you want to read.
    #Example: <table class="queryResults" border="0" cellspacing="1">
    table = soup.select_one('table', attrs={'class': 'sortable positional-spending-table'})

    # FOR DEBUGGING ###########
    # def get_all_tables(soup):
    #     return soup.find_all("table")

    # tbls = get_all_tables(soup)
    # for i, tablen in enumerate(tbls, start=1):
    #     print(i)
    #     print(tablen)
    ###########################

    ## get table column headers
    def get_table_headers(table):
        headers = []
        for th in table.find("tr") \
                    .find_all("th"):
            headers.append(th.text.strip())
        return headers

    head = get_table_headers(table)
    # print(head) ## FOR DEBUGGING ONLY

    def get_table_rows(table):    
        rows = []
        for tr in table.find_all("tr")[1:]:
            cells = []
            # grab all td tags in this table row
            tds = tr.find_all("td")
            if len(tds) == 0:
                # if no td tags, search for th tags
                # can be found especially in wikipedia tables below the table
                ths = tr.find_all("th")
                for th in ths:
                    cells.append(th.text.strip())
            else:
                # use regular td tags
                for td in tds:
                    cells.append(td.text.strip())
            rows.append(cells)
        return rows

    table_rows = get_table_rows(table)
    #print(table_rows)  ## FOR DEBUGGING ONLY

    ## save results to CSV
    def save_as_csv(table_name, headers, rows):
        pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")

    ## save results
    save_as_csv("data/nfl_team_spending_"+ search_yr +"_table", head, table_rows)


1
<table align="center" cellspacing="0" class="sortable positional-spending-table"><thead><tr><th class="sortable">Team</th><th class="sortable mobile_drop">QB</th><th class="sortable mobile_drop tablet_drop">RB</th><th class="sortable mobile_drop">WR</th><th class="sortable mobile_drop tablet_drop">TE</th><th class="sortable mobile_drop">OL</th><th class="sortable">Offense</th><th class="sortable mobile_drop">IDL</th><th class="sortable mobile_drop">EDGE</th><th class="sortable mobile_drop tablet_drop">LB</th><th class="sortable mobile_drop tablet_drop">S</th><th class="sortable mobile_drop">CB</th><th class="sortable">Defense</th></tr></thead><tbody><tr class="sortable"><td class="sortable"><a class="team-link PHI" href="/salary-cap/philadelphia-eagles/">Eagles</a></td><td class="sortable mobile_drop">$13,385,137</td><td class="sortable mobile_drop tablet_drop">$10,203,112</td><td class="sortable mobile_drop">$19,241,989</td><td class="sortable mobile_drop tablet_drop">$5,509,036</td